<a href="https://colab.research.google.com/github/shubhamk16/NLP_END_Program/blob/main/Session9/03_seq2seq_newsqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Dec 28 05:52:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!cp drive/MyDrive/newsqa-processed.csv .

In [ ]:
!ls -lah

total 241M
drwxr-xr-x 1 root root 4.0K Dec 28 05:52 .
drwxr-xr-x 1 root root 4.0K Dec 28 05:51 ..
drwxr-xr-x 1 root root 4.0K Dec 21 17:29 .config
drwx------ 4 root root 4.0K Dec 28 05:52 drive
-rw------- 1 root root 241M Dec 28 05:52 newsqa-processed.csv
drwxr-xr-x 1 root root 4.0K Dec 21 17:29 sample_data


# Load data

In [ ]:
import pandas as pd
import torchtext
import torch
import time
import random
import math

from torch import nn, optim

In [ ]:
df = pd.read_csv('newsqa-processed.csv')
df.head()

,story_qn,answer_text
0,"NEW DELHI, India (CNN) -- A high court in nort...",19
1,(CNN) -- Fighting in the volatile Sudanese reg...,Sudanese region of Darfur
2,Johannesburg (CNN) -- Miffed by a visa delay t...,Archbishop Desmond Tutu
3,(CNN) -- England international footballer Ste...,29-year-old
4,"BAGHDAD, Iraq (CNN) -- At least 6,000 Christi...",series of killings and threats by Muslim extre...


In [ ]:
print(df.shape)

(63399, 2)


In [ ]:
# Use same field for both columns since they have a shared vocabulary
TEXT = torchtext.data.Field(
    tokenize='spacy',
    lower=True,
    init_token='<sos>', 
    eos_token='<eos>'
)

fields = [('story_qn', TEXT), ('answer_text', TEXT)]

In [ ]:
# Create dataset
start = time.time()
dataset = torchtext.data.TabularDataset(
    path='newsqa-processed.csv',
    format='CSV',
    fields=fields,
    skip_header=True
)
end = time.time()
print(f'Duration: {end - start}')

Duration: 243.48657178878784


In [ ]:
# Train/val split
(train, valid) = dataset.split(split_ratio=[0.85, 0.15])
print(len(train), len(valid))

53889 9510


In [ ]:
vars(train[0])

{'answer_text': [' ',
  'attorney',
  'general',
  'jay',
  'bybee',
  'and',
  'then',
  '-',
  'princ'],
 'story_qn': ['(',
  'cnn',
  ')',
  '--',
  'she',
  'said',
  'she',
  'was',
  'a',
  'scapegoat',
  '.',
  'she',
  'said',
  'she',
  'was',
  'just',
  'following',
  'orders',
  '.',
  'she',
  'said',
  'she',
  'was',
  'demoted',
  'unfairly',
  '.',
  '\n\n',
  'retired',
  'army',
  'col',
  '.',
  'janis',
  'karpinski',
  'was',
  'one',
  'of',
  'two',
  'officers',
  'punished',
  'over',
  'abu',
  'ghraib',
  '.',
  '\n\n',
  'now',
  ',',
  'retired',
  'army',
  'col',
  '.',
  'janis',
  'karpinski',
  'can',
  'say',
  ':',
  'i',
  'told',
  'you',
  'so',
  '.',
  '\n\n',
  'karpinski',
  'was',
  'one',
  'of',
  'two',
  'officers',
  'punished',
  'over',
  'the',
  'aggressive',
  'interrogations',
  'at',
  'the',
  'notorious',
  'abu',
  'ghraib',
  'prison',
  'in',
  'iraq',
  '.',
  'pictures',
  'of',
  'detainees',
  'caused',
  'outrage',
  'a

In [ ]:
MAX_VOCAB_SIZE = 10_000

TEXT.build_vocab(train, max_size=MAX_VOCAB_SIZE)

print(f'Size of vocab: {len(TEXT.vocab)}')

Size of vocab: 10004


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator = torchtext.data.BucketIterator.splits(
    (train, valid),
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.story_qn),
    device = device
)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(TEXT.vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10004, 128)
    (rnn): GRU(128, 256)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(10004, 128)
    (rnn): GRU(384, 256)
    (fc_out): Linear(in_features=640, out_features=10004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,763,092 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.story_qn
        trg = batch.answer_text
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.story_qn
            trg = batch.answer_text

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 3m 16s
	Train Loss: 5.656 | Train PPL: 286.108
	 Val. Loss: 5.529 |  Val. PPL: 251.861
Epoch: 02 | Time: 3m 18s
	Train Loss: 5.323 | Train PPL: 204.921
	 Val. Loss: 5.529 |  Val. PPL: 251.859
Epoch: 03 | Time: 3m 17s
	Train Loss: 5.231 | Train PPL: 187.027
	 Val. Loss: 5.531 |  Val. PPL: 252.452
Epoch: 04 | Time: 3m 18s
	Train Loss: 5.148 | Train PPL: 172.007
	 Val. Loss: 5.536 |  Val. PPL: 253.676
Epoch: 05 | Time: 3m 19s
	Train Loss: 5.084 | Train PPL: 161.445
	 Val. Loss: 5.542 |  Val. PPL: 255.128
Epoch: 06 | Time: 3m 19s
	Train Loss: 5.027 | Train PPL: 152.407
	 Val. Loss: 5.551 |  Val. PPL: 257.422
Epoch: 07 | Time: 3m 19s
	Train Loss: 4.978 | Train PPL: 145.205
	 Val. Loss: 5.577 |  Val. PPL: 264.146
Epoch: 08 | Time: 3m 20s
	Train Loss: 4.936 | Train PPL: 139.250
	 Val. Loss: 5.576 |  Val. PPL: 264.118
Epoch: 09 | Time: 3m 19s
	Train Loss: 4.861 | Train PPL: 129.111
	 Val. Loss: 5.584 |  Val. PPL: 266.152
Epoch: 10 | Time: 3m 18s
	Train Loss: 4.826 | Train PPL